In [ ]:
import sqlite3
import pandas as pd
import requests
from astropy.io import fits
from astropy.table import Table
import numpy as np
import pyarrow as pa
import pyarrow.feather as feather
from upath import UPath

In [3]:
def urls_from_query(tbl, suffix="psfcat.fits", base='https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci'):
    filefracday_arr = np.array(tbl['filefracday'], dtype=np.str_)
    # print(filefracday_arr)
    YYYY_arr = [s[:4] for s in filefracday_arr]
    # print(YYYY_arr)
    MMDD_arr = [s[4:8] for s in filefracday_arr]
    fracday_arr = [s[8:] for s in filefracday_arr]
    field_arr = np.strings.zfill(np.array(tbl['field'], dtype=np.str_), 6)
    filtercode_arr = np.array(tbl['filtercode'], dtype=np.str_)
    ccdid_arr = np.strings.zfill(np.array(tbl['ccdid'], dtype=np.str_), 2)
    imgtypecode_arr = np.array(tbl['imgtypecode'], dtype=np.str_)
    qid_arr = np.array(tbl['qid'], dtype=np.str_)


    filename_pieces = ['_', filefracday_arr, 
                        '_', field_arr, 
                        '_', filtercode_arr, 
                        '_c', ccdid_arr,
                        '_', imgtypecode_arr,
                        '_q', qid_arr,
                        '_', suffix]
    filename_arr = 'ztf'
    for arr in filename_pieces:
        filename_arr = np.strings.add(filename_arr, arr)
    # filename = f"ztf_{filefracday}_{field}_{filtercode}_c{ccdid}_{imgtypecode}_q{qid}_{suffix}"

    url_pieces = ['/', YYYY_arr, '/', MMDD_arr, '/', fracday_arr, '/', filename_arr]
    url_arr = base
    for arr in url_pieces:
        url_arr = np.strings.add(url_arr, arr)
    # test_url = "/".join([base, YYYY, MMDD, fracday, filename])
    return filename_arr, url_arr

In [3]:
# base='https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci'
# urls_arr = []
# for row in t_795:
#     filefracday = str(row['filefracday'])
#     YYYY, MMDD, fracday = filefracday[:4], filefracday[4:8], filefracday[8:]
#     filefracday = row['filefracday']
#     field = f"000{row['field']}"
#     filtercode = row['filtercode']
#     ccdid = row['ccdid']
#     imgtypecode = row['imgtypecode']
#     qid = row['qid']
#     suffix = "psfcat.fits"
#     filename = f"ztf_{filefracday}_{field}_{filtercode}_c{ccdid}_{imgtypecode}_q{qid}_{suffix}"
#     url = "/".join([base, YYYY, MMDD, fracday, filename])
#     urls_arr += url
# print(urls_arr[:10])

### Import field=795 query table

In [4]:
t_795 = Table.read("C:\\Users\\lupin\\OneDrive\\Desktop\\LINCC\\query_field795.tbl", format="ascii.ipac")

t_795[:5]

ra,dec,infobits,field,ccdid,qid,rcid,fid,filtercode,pid,nid,expid,itid,imgtype,imgtypecode,obsdate,obsjd,exptime,filefracday,seeing,airmass,moonillf,moonesb,maglimit,crpix1,crpix2,crval1,crval2,cd11,cd12,cd21,cd22,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,ipac_pub_date,ipac_gid
deg,deg,,,,,,,,,,,,,,,d,s,,arcsec,,,,mag,pix,pix,deg,deg,deg / pix,deg / pix,deg / pix,deg / pix,deg,deg,deg,deg,deg,deg,deg,deg,,
float64,float64,int64,int64,int64,int64,int64,int64,str2,int64,int64,int64,int64,str6,str1,str26,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str22,int64
245.83269706,55.49914106,0,795,11,4,43,2,zr,553202214315,553,55320221,1,object,o,2018-07-08 04:51:11.555+00,2458307.7022106,30.0,20180708202211,1.55479002,1.07299995,-0.307278991,0.0,21.2096004,1536.5,1540.5,245.8326383241,55.49917705019,-0.0002812486351572,2.752789615522e-06,-2.800402337941e-06,-0.0002812518646279,246.59621789,55.93426149,245.05389256,55.92561904,245.08585066,55.05946855,246.59430166,55.06807639,2020-06-24 00:00:00+00,2
242.53425352,56.30053076,0,795,12,1,44,1,zg,2365281794415,2365,236528179,1,object,o,2023-06-24 06:45:47.325+00,2460119.781794,30.0,20230624281782,2.05425,1.08500004,0.307837009,0.0,21.1693993,1536.5,1540.5,242.5342520805,56.30050859981,-0.0002808158925869,1.654127664556e-05,-1.644827248266e-05,-0.0002807932074442,243.27434072,56.7558546,241.70208336,56.70487486,241.81122371,55.84070717,243.34866346,55.89067207,2025-01-07 00:00:00+00,2
250.80195538,54.35608397,0,795,5,2,17,2,zr,1984425101715,1984,198442510,1,object,o,2022-06-08 10:12:09.306+00,2459738.9251042,30.0,20220608425093,1.55728996,1.278,0.581937015,0.0,21.0077991,1536.5,1540.5,250.8020188547,54.35603348862,-0.000280773293904,-1.735283543558e-05,1.733416982548e-05,-0.0002807575610519,251.59570197,54.75914436,250.10016716,54.81285519,250.02386296,53.94784604,251.48872371,53.8950516,2024-01-08 00:00:00+00,2
245.90216829,55.47859784,0,795,11,4,43,2,zr,1993360524315,1993,199336052,1,object,o,2022-06-17 08:39:09.449+00,2459747.8605208,30.0,20220617360509,1.47934997,1.16600001,-0.879687011,0.0,20.7625999,1536.5,1540.5,245.9021555806,55.47860887891,-0.0002812854613218,2.432813976849e-06,-2.458699951409e-06,-0.0002812854971088,246.66638522,55.91313243,245.12457239,55.90568322,245.15464358,55.0394991,246.6625114,55.04696949,2024-01-08 00:00:00+00,2
251.01515916,56.27221105,0,795,9,2,33,2,zr,2368286963315,2368,236828696,1,object,o,2023-06-27 06:53:14.125+00,2460122.7869676,30.0,20230627286956,2.46930003,1.09500003,0.594126999,0.0,20.4377003,1536.5,1540.5,251.0152314038,56.27217149944,-0.0002806837815152,-1.832906183275e-05,1.825629882693e-05,-0.0002806514995715,251.85119958,56.67346778,250.28109949,56.7301045,250.19672639,55.86530423,251.73228306,55.80989827,2023-09-07 00:00:00+00,1


#### filter for a single set of parameters
filtercode, ccdid, imgtypecode, qid\
suffix is psfcat.fits

In [5]:
# syntax: filtered = t[(t['flux'] > 1.0) & (t['z'] < 0.2)]
# parentheses are necessary

In [5]:
t_795_zr_c01_o_q1 = t_795[(t_795['filtercode'] == 'zr') & (t_795['qid'] == 1) & (t_795['ccdid'] == 1) & (t_795['imgtypecode'] == 'o')]
# f"ztf_{filefracday}_{field}_{filtercode}_c{ccdid}_{imgtypecode}_q{qid}_{suffix}"
len(t_795_zr_c01_o_q1), len(t_795)

(1694, 235110)

In [6]:
t_795_zr_c01_o_q1[:2]

ra,dec,infobits,field,ccdid,qid,rcid,fid,filtercode,pid,nid,expid,itid,imgtype,imgtypecode,obsdate,obsjd,exptime,filefracday,seeing,airmass,moonillf,moonesb,maglimit,crpix1,crpix2,crval1,crval2,cd11,cd12,cd21,cd22,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,ipac_pub_date,ipac_gid
deg,deg,,,,,,,,,,,,,,,d,s,,arcsec,,,,mag,pix,pix,deg,deg,deg / pix,deg / pix,deg / pix,deg / pix,deg,deg,deg,deg,deg,deg,deg,deg,,
float64,float64,int64,int64,int64,int64,int64,int64,str2,int64,int64,int64,int64,str6,str1,str26,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str22,int64
251.96900978,52.44297858,0,795,1,1,0,2,zr,519323200015,519,51932320,1,object,o,2018-06-04 07:45:25.127+00,2458273.823206,30.0,20180604323194,1.99293005,1.07599998,-0.729726017,0.0,20.7616005,1536.5,1540.5,251.9689868048,52.44294330486,-0.0002803325071178,-2.216581152326e-05,2.228680662349e-05,-0.0002803232997059,252.73809546,52.83777802,251.31163838,52.90705663,251.21337023,52.04293188,252.61250574,51.97531037,2019-12-11 00:00:00+00,2
251.95901291,52.42696114,0,795,1,1,0,2,zr,547182010015,547,54718201,1,object,o,2018-07-02 04:22:06.635+00,2458301.6820139,30.0,20180702181968,2.43827009,1.10000002,-0.864785016,0.0,20.4754009,1536.5,1540.5,251.9589799988,52.42692651577,-0.0002803299413277,-2.221211384881e-05,2.231619583144e-05,-0.0002803185512301,252.72795045,52.82171009,251.30198574,52.89108034,251.20347449,52.02697318,252.60218223,51.95924548,2020-06-24 00:00:00+00,2


In [7]:
filenames, urls = urls_from_query(t_795_zr_c01_o_q1)

In [9]:
# for filename, url in zip(filenames, urls):
#     filename = f"t_795_zr_c01_o_q1/{filename}"
#     !curl -sSf -o {filename} {url}

# output: 6 files note found ("curl: (22) The requested URL returned error: 404")

### Unit test: save metadata for use in overwritten fitsreader 

In [23]:
filename = r"t_795_zr_c01_o_q1\ztf_20190613249421_000795_zr_c01_o_q1_psfcat.feather"
row = t_795_zr_c01_o_q1[t_795_zr_c01_o_q1['filefracday'] == 20190613249421]
row

ra,dec,infobits,field,ccdid,qid,rcid,fid,filtercode,pid,nid,expid,itid,imgtype,imgtypecode,obsdate,obsjd,exptime,filefracday,seeing,airmass,moonillf,moonesb,maglimit,crpix1,crpix2,crval1,crval2,cd11,cd12,cd21,cd22,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,ipac_pub_date,ipac_gid
deg,deg,,,,,,,,,,,,,,,d,s,,arcsec,,,,mag,pix,pix,deg,deg,deg / pix,deg / pix,deg / pix,deg / pix,deg,deg,deg,deg,deg,deg,deg,deg,,
float64,float64,int64,int64,int64,int64,int64,int64,str2,int64,int64,int64,int64,str6,str1,str26,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str22,int64
251.98365414,52.37150135,0,795,1,1,0,2,zr,893249450015,893,89324945,1,object,o,2019-06-13 05:59:13.102+00,2458647.749456,30.0,20190613249421,1.64551997,1.08800006,0.81556201,0.0,20.5263996,1536.5,1540.5,251.9836337285,52.37149818724,-0.0002803198686738,-2.231517323406e-05,2.243884291977e-05,-0.0002803307680752,252.75185019,52.76604081,251.32776692,52.83575383,251.22881667,51.97168531,252.62572927,51.90366798,2020-12-09 00:00:00+00,2


In [24]:
# convert astropy to pyarrow: cast to pd df then to pyarrow
df = row.to_pandas()
paTable = pa.Table.from_pandas(df, preserve_index=False)
print(paTable['field'])
df

[
  [
    795
  ]
]


,ra,dec,infobits,field,ccdid,qid,rcid,fid,filtercode,pid,...,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,ipac_pub_date,ipac_gid
0,251.983654,52.371501,0,795,1,1,0,2,zr,893249450015,...,252.75185,52.766041,251.327767,52.835754,251.228817,51.971685,252.625729,51.903668,2020-12-09 00:00:00+00,2


In [25]:
# save metadata with pyarrow feather
filepath = UPath(filename)
meta_filename = (filepath.parent/("meta_" + filepath.name)).with_suffix(".feather")

feather.write_feather(paTable, meta_filename)

In [26]:
feather_test = feather.read_table(meta_filename)
feather_test

pyarrow.Table
ra: double
dec: double
infobits: int64
field: int64
ccdid: int64
qid: int64
rcid: int64
fid: int64
filtercode: string
pid: int64
nid: int64
expid: int64
itid: int64
imgtype: string
imgtypecode: string
obsdate: string
obsjd: double
exptime: double
filefracday: int64
seeing: double
airmass: double
moonillf: double
moonesb: double
maglimit: double
crpix1: double
crpix2: double
crval1: double
crval2: double
cd11: double
cd12: double
cd21: double
cd22: double
ra1: double
dec1: double
ra2: double
dec2: double
ra3: double
dec3: double
ra4: double
dec4: double
ipac_pub_date: string
ipac_gid: int64
----
ra: [[251.98365414]]
dec: [[52.37150135]]
infobits: [[0]]
field: [[795]]
ccdid: [[1]]
qid: [[1]]
rcid: [[0]]
fid: [[2]]
filtercode: [["zr"]]
pid: [[893249450015]]
...

In [27]:
feather_test['expid'][0], feather_test['rcid'][0], feather_test['obsjd'][0], feather_test['field'][0]

(<pyarrow.Int64Scalar: 89324945>,
 <pyarrow.Int64Scalar: 0>,
 <pyarrow.DoubleScalar: 2458647.749456>,
 <pyarrow.Int64Scalar: 795>)

### Save metadata for all urls

In [33]:
for filename in filenames:
	row = t_795_zr_c01_o_q1[t_795_zr_c01_o_q1['filefracday'] == int(filename.split("_")[1])]
	df = row.to_pandas()
	paTable = pa.Table.from_pandas(df, preserve_index=False)
	filepath = UPath(filename)
	meta_filename =  't_795_zr_c01_o_q1'/UPath("meta_" + filepath.name).with_suffix(".feather")
	if (len(paTable) > 0):
		feather.write_feather(paTable, meta_filename)
	else:
		print("Not written to:", meta_filename)

In [12]:
feather.read_table('t_795_zr_c01_o_q1/meta_ztf_20180321399988_000795_zr_c01_o_q1_psfcat.feather')

pyarrow.Table
ra: double
dec: double
infobits: int64
field: int64
ccdid: int64
qid: int64
rcid: int64
fid: int64
filtercode: null
pid: int64
nid: int64
expid: int64
itid: int64
imgtype: null
imgtypecode: null
obsdate: null
obsjd: double
exptime: double
filefracday: int64
seeing: double
airmass: double
moonillf: double
moonesb: double
maglimit: double
crpix1: double
crpix2: double
crval1: double
crval2: double
cd11: double
cd12: double
cd21: double
cd22: double
ra1: double
dec1: double
ra2: double
dec2: double
ra3: double
dec3: double
ra4: double
dec4: double
ipac_pub_date: null
ipac_gid: int64
----
ra: []
dec: []
infobits: []
field: []
ccdid: []
qid: []
rcid: []
fid: []
filtercode: []
pid: []
...

In [ ]:
feather.read_table('ztf_dr23\meta_ztf_20180608284988_000795_zr_c01_o_q1_psfcat.feather')

pyarrow.Table
ra: double
dec: double
infobits: int64
field: int64
ccdid: int64
qid: int64
rcid: int64
fid: int64
filtercode: null
pid: int64
nid: int64
expid: int64
itid: int64
imgtype: null
imgtypecode: null
obsdate: null
obsjd: double
exptime: double
filefracday: int64
seeing: double
airmass: double
moonillf: double
moonesb: double
maglimit: double
crpix1: double
crpix2: double
crval1: double
crval2: double
cd11: double
cd12: double
cd21: double
cd22: double
ra1: double
dec1: double
ra2: double
dec2: double
ra3: double
dec3: double
ra4: double
dec4: double
ipac_pub_date: null
ipac_gid: int64
----
ra: []
dec: []
infobits: []
field: []
ccdid: []
qid: []
rcid: []
fid: []
filtercode: []
pid: []
...